In [12]:
import numpy as np
import numpy.linalg as lin
from typing import List, Callable, Tuple
from fractions import Fraction

def hypercube(dims: int) -> List[Tuple]:
    if dims == 0:
        return [()]
    for elem in hypercube(dims - 1):
        yield (0, *elem.coords)
        yield (1, *elem.coords)

# Spline representation: [order+1]**dims
def spline_new(dims: int, order: int):
    return np.ones((order+1)**dims)

def spline_iterator(dims: int, order: int):
    if dims == 0:
        #print("A")
        yield []
    else:
        for elem in spline_iterator(dims-1, order):
            for i in range(order+1):
                yield elem + [i]
                
def spline_pos(dims: int, order: int, orders: List) -> int:
    assert len(orders) == dims
    assert all([o <= order for o in orders])
    assert all([o >= 0 for o in orders])
    return sum([o * (order+1)**d for d, o in enumerate(orders)])

def hypercube(dims: int):
    return spline_iterator(dims, 1)

def derivative_at(dims: int, order: int, spline: np.array, derivative: List[int], point: List[int]) -> np.array:
    new_spline = spline[:]
    for orders in spline_iterator(dims, order):
        pos = spline_pos(dims, order, orders)
        eff_orders = orders[:]
        factor = 1
        for der in derivative:
            if eff_orders[der] >= 0:
                factor *= eff_orders[der]
                eff_orders[der] -= 1
        for dim in range(dims):
            if point[dim] == 0 and eff_orders[dim] > 0:
                factor = 0
        new_spline[pos] = spline[pos] * factor
    return new_spline

def choices(n: int, k: int, rest = []):
    if n == k:
        yield rest + list(range(n))
    elif k == 0:
        yield rest[:]
    else:
        yield from choices(n - 1, k, rest)
        yield from choices(n - 1, k - 1, rest + [n - 1])

def derivatives(dims: int, order: int) -> List[List[int]]:
    # get all the different choices
    # get them 3: one, 5: twice, 7 thrice, (order - 1) / 2 times
    if order == 1:
        return [[]]
    else:
        repeat = (order - 1) // 2
        result = set([()])
        once = [elem for i in range(dims+1) for elem in choices(dims, i)]
        for i in range(repeat):
            new_result = set()
            for elem1 in once:
                for elem2 in result:
                    new_result.add(tuple(sorted(elem1 + list(elem2))))
            result = new_result
        return list(sorted(map(list, result), key=lambda a: ''.join(map(repr, a)), reverse=True))

def fit_coeffs(dims: int, order: int) -> np.array:
    entries = []
    ordering = []
    for point in hypercube(dims):
        for der in derivatives(dims, order):
            #print(point, der)
            spline = spline_new(dims, order)
            spline = derivative_at(dims, order, spline, der, point)
            #spline = differentiate(dims, order, spline, der)
            #spline = evaluate(dims, order, spline, point)
            entries.append(spline)
            ordering.append((point, der))
    return np.array(entries), ordering

def gaussj(A):
    N = A.shape[0]
    indxc = np.zeros(N, dtype=int)
    indxr = np.zeros(N, dtype=int)
    ipiv = np.zeros(N, dtype=int)
    for i in range(N):
        big = 0
        for j in range(N):
            if ipiv[j] != 1:
                for k in range(N):
                    if ipiv[k] == 0:
                        if abs(A[j, k]) > big:
                            big = abs(A[j, k])
                            irow = j
                            icol = k
        assert big > 0
        ipiv[icol] += 1
        if irow != icol:
            for l in range(N):
                A[irow, l], A[icol, l] = A[icol, l], A[irow, l]
        indxr[i] = irow
        indxc[i] = icol
        assert A[icol, icol] != 0
        pivinv = 1. / A[icol, icol]
        A[icol, icol] = 1
        for l in range(N):
            A[icol, l] *= pivinv
        for ll in range(N):
            if ll != icol:
                dum = A[ll, icol]
                A[ll, icol] = 0
                for l in range(N):
                    A[ll, l] -= A[icol, l] * dum
    for ll in range(N):
        l = N - 1 - ll
        if indxr[l] != indxc[l]:
            for k in range(N):
                A[k, indxr[l]], A[k, indxc[l]] = A[k, indxc[l]], A[k, indxr[l]]
    return A


def rat_gcd(a, b):
    while b != 0:
        a, b = b, a % b,
    return a

def rat_norm(n, d):
    g = rat_gcd(abs(n), abs(d))
    n = n // g
    d = d // g
    if n == 0:
        d = 1
    if d < 0:
        n, d = -n, -d
    return n, d

def rat_add_mul(na, da, nb, db, nc, dc):
    #return na / da + nb * nc / db / dc, 1
    na, da = rat_norm(na, da)
    nb, db = rat_norm(nb, db)
    nc, dc = rat_norm(nc, dc)
    nd = nb * nc
    dd = db * dc
    nd, dd = rat_norm(nd, dd)
    ne = na * dd + nd * da
    de = dd * da
    ne, de = rat_norm(ne, de)
    return ne, de


In [13]:
def gaussj(A):
    A = 1 * A
    N = A.shape[0]
    indxc = np.zeros(N, dtype=int)
    indxr = np.zeros(N, dtype=int)
    ipiv = np.zeros(N, dtype=int)
    AA = np.ones((N, N, 2), dtype=int)
    AA[:, :, 0] = A
    A = AA
    for i in range(N):
        big = 0
        for j in range(N):
            if ipiv[j] != 1:
                for k in range(N):
                    if ipiv[k] == 0:
                        if abs(A[j, k, 0] / A[j, k, 1]) > big:
                            big = abs(A[j, k, 0] / A[j, k, 1])
                            irow = j
                            icol = k
        assert big > 0
        ipiv[icol] += 1
        if irow != icol:
            for l in range(N):
                for c in range(2):
                    A[irow, l, c], A[icol, l, c] = A[icol, l, c], A[irow, l, c]
        indxr[i] = irow
        indxc[i] = icol
        assert A[icol, icol, 0] != 0
        pivinvden = A[icol, icol, 0]
        pivinvnum = A[icol, icol, 1]
        A[icol, icol, 0] = 1
        A[icol, icol, 1] = 1
        for l in range(N):
            A[icol, l, 0], A[icol, l, 1] = rat_add_mul(0, 1, A[icol, l, 0], A[icol, l, 1], pivinvnum, pivinvden)
        for ll in range(N):
            if ll != icol:
                dumnum = A[ll, icol, 0]
                dumden = A[ll, icol, 1]
                A[ll, icol, 0] = 0
                A[ll, icol, 1] = 1
                for l in range(N):
                    #A[ll, l, 0] -= A[icol, l, 0] * dum
                    A[ll, l, 0], A[ll, l, 1] = rat_add_mul(A[ll, l, 0], A[ll, l, 1], -A[icol, l, 0], A[icol, l, 1], dumnum, dumden)
    for ll in range(N):
        l = N - 1 - ll
        if indxr[l] != indxc[l]:
            for k in range(N):
                for c in range(2):
                    A[k, indxr[l], c], A[k, indxc[l], c] = A[k, indxc[l], c], A[k, indxr[l], c]
    print(np.max(A[:, :, 1]))
    return A[:, :, 0] / A[:, :, 1]

C = gaussj(A)
np.max(B - C)
C

AssertionError: 

In [14]:
A, O = fit_coeffs(1, 5)
B = lin.inv(A)
C = gaussj(A)
B, O

2


(array([[  0. ,   0. ,   1. ,   0. ,   0. ,   0. ],
        [  0. ,   1. ,   0. ,   0. ,   0. ,   0. ],
        [  0.5,   0. ,   0. ,   0. ,   0. ,   0. ],
        [ -1.5,  -6. , -10. ,   0.5,  -4. ,  10. ],
        [  1.5,   8. ,  15. ,  -1. ,   7. , -15. ],
        [ -0.5,  -3. ,  -6. ,   0.5,  -3. ,   6. ]]),
 [([0], [0, 0]), ([0], [0]), ([0], []), ([1], [0, 0]), ([1], [0]), ([1], [])])

In [165]:
def rat_gcd(a, b):
    while b != 0:
        a, b = b, a % b,
    return a

def rat_norm(n, d):
    g = rat_gcd(abs(n), abs(d))
    n /= g
    d /= g
    if n == 0:
        d = 1
    if d < 0:
        n, d = -n, -d
    return n, d

def rat_sub_mul(na, da, nb, db, nc, dc):
    return na / da - nb * nc / db / dc, 1
    na, da = rat_norm(na, da)
    nb, db = rat_norm(nb, db)
    nc, dc = rat_norm(nc, dc)
    nd = nb * nc
    dd = db * dc
    nd, dd = rat_norm(nd, dd)
    ne = na * dd - nd * da
    de = dd * da
    ne, de = rat_norm(ne, de)
    return ne, de

def rat_gaussj(A):
    N = A.shape[0]
    indxc = np.zeros(N, dtype=int)
    indxr = np.zeros(N, dtype=int)
    ipiv = np.zeros(N, dtype=int)
    Anum = 1 * A
    Aden = 1 + 0 * A
    for i in range(N):
        big = 0
        for j in range(N):
            if ipiv[j] != 1:
                for k in range(N):
                    if ipiv[k] == 0:
                        if abs(Anum[j, k] / Aden[j, k]) > big:
                            big = abs(Anum[j, k] / Aden[j, k])
                            irow = j
                            icol = k
        assert big > 0
        ipiv[icol] += 1
        if irow != icol:
            for l in range(N):
                Anum[irow, l], Anum[icol, l] = Anum[icol, l], Anum[irow, l]
                Aden[irow, l], Aden[icol, l] = Aden[icol, l], Aden[irow, l]
        indxr[i] = irow
        indxc[i] = icol
        assert Anum[icol, icol] != 0
        pivinvnum = Aden[icol, icol]
        pivinvden = Anum[icol, icol] #1 / A[icol, icol]
        Anum[icol, icol] = 1
        Aden[icol, icol] = 1
        for l in range(N):
            Anum[icol, l], Aden[icol, l] = rat_sub_mul(0, 1, Anum[icol, l], Aden[icol, l], pivinvnum, pivinvden)
        for ll in range(N):
            if ll != icol:
                dumnum = Anum[ll, icol]
                dumden = Aden[ll, icol]
                Anum[ll, icol] = 0
                Aden[ll, icol] = 1
                for l in range(N):
                    Anum[ll, l], Aden[ll, l] = rat_sub_mul(Anum[ll, l], Aden[ll, l], Anum[icol, l], Aden[icol, l], dumnum, dumden)
    for ll in range(N):
        l = N - 1 - ll
        if indxr[l] != indxc[l]:
            for k in range(N):
                Anum[k, indxr[l]], Anum[k, indxc[l]] = Anum[k, indxc[l]], Anum[k, indxr[l]]
                Aden[k, indxr[l]], Aden[k, indxc[l]] = Aden[k, indxc[l]], Aden[k, indxr[l]]
    return Anum / Aden

In [85]:
derivatives(2, 8)

[[],
 [1],
 [1, 1],
 [1, 1, 1],
 [0],
 [0, 1],
 [0, 1, 1],
 [0, 1, 1, 1],
 [0, 0],
 [0, 0, 1],
 [0, 0, 1, 1],
 [0, 0, 1, 1, 1],
 [0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 0, 1, 1],
 [0, 0, 0, 1, 1, 1]]

In [91]:
from sympy.matrices import Matrix

In [259]:
np.zeros(5, dtype=int)

array([0, 0, 0, 0, 0])

In [99]:
#evaluate(1, 3, spline_new(1, 3), (0,))
#differentiate(1, 3, spline_new(1, 3), [0, 0, 0, 0, 0])

array([ 0.,  0.,  0.,  0.])

In [ ]:

def rat_gaussj(A):
    N = A.shape[0]
    indxc = np.zeros(N, dtype=int)
    indxr = np.zeros(N, dtype=int)
    ipiv = np.zeros(N, dtype=int)
    A = np.array(A, dtype=int)
    Anum = 1 * A
    Aden = 1 + 0 * A
    for i in range(N):
        big = 0
        for j in range(N):
            if ipiv[j] != 1:
                for k in range(N):
                    if ipiv[k] == 0:
                        if abs(Anum[j, k] / Aden[j, k]) > big:
                            big = abs(Anum[j, k] / Aden[j, k])
                            irow = j
                            icol = k
        assert big > 0
        ipiv[icol] += 1
        if irow != icol:
            for l in range(N):
                Anum[irow, l], Anum[icol, l] = Anum[icol, l], Anum[irow, l]
                Aden[irow, l], Aden[icol, l] = Aden[icol, l], Aden[irow, l]
        indxr[i] = irow
        indxc[i] = icol
        assert Anum[icol, icol] != 0
        pivinvnum = Aden[icol, icol]
        pivinvden = Anum[icol, icol] #1 / A[icol, icol]
        Anum[icol, icol] = 1
        Aden[icol, icol] = 1
        for l in range(N):
            Anum[icol, l], Aden[icol, l] = rat_sub_mul(0, 1, -Anum[icol, l], Aden[icol, l], pivinvnum, pivinvden)
        for ll in range(N):
            if ll != icol:
                dumnum = Anum[ll, icol]
                dumden = Aden[ll, icol]
                Anum[ll, icol] = 0
                Aden[ll, icol] = 1
                for l in range(N):
                    Anum[ll, l], Aden[ll, l] = rat_sub_mul(Anum[ll, l], Aden[ll, l], Anum[icol, l], Aden[icol, l], dumnum, dumden)
    for ll in range(N):
        l = N - 1 - ll
        if indxr[l] != indxc[l]:
            for k in range(N):
                Anum[k, indxr[l]], Anum[k, indxc[l]] = Anum[k, indxc[l]], Anum[k, indxr[l]]
                Aden[k, indxr[l]], Aden[k, indxc[l]] = Aden[k, indxc[l]], Aden[k, indxr[l]]
    return np.array(Anum, dtype=float) / Aden


def rat2_gaussj(A):
    N = A.shape[0]
    indxc = np.zeros(N, dtype=int)
    indxr = np.zeros(N, dtype=int)
    ipiv = np.zeros(N, dtype=int)
    Aden = 1 + 0 * A
    for i in range(N):
        big = 0
        for j in range(N):
            if ipiv[j] != 1:
                for k in range(N):
                    if ipiv[k] == 0:
                        if abs(A[j, k] / Aden[j, k]) > big:
                            big = abs(A[j, k] / Aden[j, k])
                            irow = j
                            icol = k
        assert big > 0
        ipiv[icol] += 1
        if irow != icol:
            for l in range(N):
                A[irow, l], A[icol, l] = A[icol, l], A[irow, l]
                Aden[irow, l], Aden[icol, l] = Aden[icol, l], Aden[irow, l]
        indxr[i] = irow
        indxc[i] = icol
        assert A[icol, icol] != 0
        pivinv = 1. / A[icol, icol]
        A[icol, icol] = 1
        Aden[icol, icol] = 1
        for l in range(N):
            A[icol, l] *= pivinv
        for ll in range(N):
            if ll != icol:
                dum = A[ll, icol]
                A[ll, icol] = 0
                Aden[ll, icol] = 1
                for l in range(N):
                    A[ll, l] -= A[icol, l] * dum
    for ll in range(N):
        l = N - 1 - ll
        if indxr[l] != indxc[l]:
            for k in range(N):
                A[k, indxr[l]], A[k, indxc[l]] = A[k, indxc[l]], A[k, indxr[l]]
                Aden[k, indxr[l]], Aden[k, indxc[l]] = Aden[k, indxc[l]], Aden[k, indxr[l]]
    return A / Aden


def rat2_gaussj(A):
    N = A.shape[0]
    indxc = np.zeros(N, dtype=int)
    indxr = np.zeros(N, dtype=int)
    ipiv = np.zeros(N, dtype=int)
    for i in range(N):
        big = 0
        for j in range(N):
            if ipiv[j] != 1:
                for k in range(N):
                    if ipiv[k] == 0:
                        if abs(A[j, k]) > big:
                            big = abs(A[j, k])
                            irow = j
                            icol = k
        assert big > 0
        ipiv[icol] += 1
        if irow != icol:
            for l in range(N):
                A[irow, l], A[icol, l] = A[icol, l], A[irow, l]
        indxr[i] = irow
        indxc[i] = icol
        assert A[icol, icol] != 0
        pivinv = 1. / A[icol, icol]
        A[icol, icol] = 1
        for l in range(N):
            A[icol, l] *= pivinv
        for ll in range(N):
            if ll != icol:
                dum = A[ll, icol]
                A[ll, icol] = 0
                for l in range(N):
                    A[ll, l] -= A[icol, l] * dum
    for ll in range(N):
        l = N - 1 - ll
        if indxr[l] != indxc[l]:
            for k in range(N):
                A[k, indxr[l]], A[k, indxc[l]] = A[k, indxc[l]], A[k, indxr[l]]
    return A


C = rat_gaussj(A)
print(B)
print(C)
#np.nanmax(np.abs(C - B) / np.maximum(np.abs(C), np.abs(B)))
B-C
#print(A.shape)
#C = [[Fraction(B[i, j]).limit_denominator() for j in range(A.shape[0])] for i in range(A.shape[0])]
#max([Fraction(B[i, j]).limit_denominator().denominator for j in range(A.shape[0]) for i in range(A.shape[0])])